In [ ]:
# pdf webscrapping and text extraction

# get the pdf files or url from the web
import requests

# for tree traversal scraping in webpage
from bs4 import BeautifulSoup

# input output operations
import io

# getting information from the pdfs
from PyPDF2 import PdfReader

%pip install xhtml2pdf requests




## Extraction of url text 001:
Front page with 730 links, where each link directs you to another page with a specific research paper.
Code below extracts text from the second webpage. However, it is not able to extract text from the pdf that are linked in the second page. There are 2 different links for downloading the same pdf. 
- DOI and 
- ShareIt


```<p>
DOI: 
<a href="https://doi.org/10.1007/978-3-031-43990-2_43">https://doi.org/10.1007/978-3-031-43990-2_43</a>
</p>


<p>
SharedIt: 
<a href="https://rdcu.be/dnwLY">https://rdcu.be/dnwLY</a>
<br><br>
</p>
```

In [ ]:
!pip install lxml

In [245]:
import requests 
from bs4 import BeautifulSoup 
import regex as re
from urllib.request import urlopen
from urllib import request as urllib2

# specify the URL of the archive here 
archive_url = "http://conferences.miccai.org/2023/papers/"
url = "http://conferences.miccai.org"


list_of_pdfs = []
# create response object 
r = requests.get(archive_url) 

# create beautiful-soup object 
soup = BeautifulSoup(r.content,'html')

# find all links on web-page 
links = soup.findAll('a') 

# filter links ending with .html 
list_of_urls = [url + link['href'] for link in links if link['href'].endswith('html')] 
# print(len(paper_links)) gives us 730 papers in total


# now that we have all the links to the papers, we can go through each paper and find the pdf link
for link in list_of_urls:
	# create response object per link from the total list of links
	r = requests.get(link) 
	soup = BeautifulSoup(r.content,'html')

	# find all urls with 'rdcu.be' which contains the specific paper
	links = soup.findAll(attrs={'href': re.compile("rdcu.be")})
	pdfs = [link['href'] for link in links]	
	list_of_pdfs.extend(pdfs)

print(len(list_of_pdfs))


2024-02-11 16:32:02 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): conferences.miccai.org:80
2024-02-11 16:32:02 [urllib3.connectionpool] DEBUG: http://conferences.miccai.org:80 "GET /2023/papers/ HTTP/1.1" 200 44307
2024-02-11 16:32:03 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): conferences.miccai.org:80
2024-02-11 16:32:03 [urllib3.connectionpool] DEBUG: http://conferences.miccai.org:80 "GET /2023/papers/001-Paper0829.html HTTP/1.1" 200 51950
2024-02-11 16:32:03 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): conferences.miccai.org:80
2024-02-11 16:32:03 [urllib3.connectionpool] DEBUG: http://conferences.miccai.org:80 "GET /2023/papers/002-Paper1655.html HTTP/1.1" 200 35534
2024-02-11 16:32:03 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): conferences.miccai.org:80
2024-02-11 16:32:03 [urllib3.connectionpool] DEBUG: http://conferences.miccai.org:80 "GET /2023/papers/003-Paper1307.html HTTP/1.1" 200 24103
20

730


In [ ]:
for link in list_of_pdfs:
	# create response object per link from the total list of links
	r = requests.get(link) 
	soup = BeautifulSoup(r.content,'html')
	# find all urls with 'rdcu.be' which contains the specific paper
	links = soup.findAll(attrs={'href': re.compile("/content/pdf")})
	final_list = [link['content'] for link in links]	

print(len(final_list))  

In [ ]:
for pdf in list_of_pdfs:
    filename = pdf.split("/")[-1]
    res = requests.get(pdf)
    pdf = open("pdfs/" + filename, 'wb')
    pdf.write(res.content)
    pdf.close() 